# Managing custom Jupyter environments with Singularity
---
In this tutorial we will cover the basic workflow for managing custom software environments for Jupyter Notebooks using Singularity.

Singularity is a developing platform, so version matters **a lot**. The version we will be using is the latest development HEAD of 2.3.

In [3]:
singularity --version

2.3-HEAD.gadf5259


## Pull the base image from ~~SingularityHub~~ DockerHub
We maintain a base image on SingularityHub for running Jupyter* (https://singularity-hub.org/collections/440/). This image contains the minimum dependencies and configuration needed to run containerized Notebooks (standalone or JupyterHub-spawned), and is intended to serve as a base for user-built software environments.

At this time though bootstrapping from SingularityHub is still an upcoming feature (https://github.com/singularityware/singularity/issues/833) so we will instead be using the `jupyter/base-notebook` docker conatainer as a base image. The base-notebook is provided by the Jupyter Docker Stacks project (https://github.com/jupyter/docker-stacks), which provides pre-built stacks ready to be run standalone or behind JupyterHub.

### A basic pull
Start by pulling the Jupyter base image from SingularityHub. We specifically want commit _ae885c0a6226_, so we'll specify that by adding the `:ae885c0a6226` tag to the end of the repo path:

In [1]:
singularity pull --name "jupyter-base.img" docker://jupyter/base-notebook:ae885c0a6226

Initializing Singularity image subsystem
Opening image file: jupyter-base.img
Creating 892MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-base.img
Docker image path: index.docker.io/jupyter/base-notebook:ae885c0a6226
Cache folder set to /home/vagrant/.singularity/docker
Importing: base Singularity environment
Importing: /home/vagrant/.singularity/docker/sha256:e0a742c2abfd5e2a6f8ed15b1c78e873cf9559b96a04204daf6de5df01e3124c.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:486cb8339a27635fa93dc47aa0c689326a0a7cce388966d16daf8d265436cf7f.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:dc6f0d824617ad8a5d1163a5b2084814665dd83156317ad06ccf14deb517a053.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:4f7a5649a30e3f318ce5d7e4dbcbbeb6c0938c4cbae4d4a641fe910562ff4978.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:672363445ad2c734e29221a6b47f4e614b5adc8a3cdca3364f62db2ed2bdff0c.tar.gz
Importing: /home/vagrant/.si

There it is! Your container is good to go.

In [2]:
singularity exec -e jupyter-base.img jupyter -h

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the jupyter command's version and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json

Available subcommands: bundlerextension kernelspec lab labextension labhub
migrate nbconvert nbextension notebook run serverextension troubleshoot trust


## Customizing the base image
The base image is meant to capture the _minimum_ config and dependencies to run Jupyter Notebooks. Here we detail how to customize the base image to better suit your needs.

### Resize the image
This image uses the default size set by Singularity _(image size + 200M of padding)_ which is great for quick builds and pulls, but it is likely you'll need more space to accommodate your custom software stack.

In [3]:
ls -lsah | grep jupyter-base.img

669M -rwxr-xr-x.  1 vagrant vagrant 893M Sep 27 15:41 jupyter-base.img


When pulling from a Docker registry, you can use the `--size` flag to specify the built image size. Notice that Singularity isn't grabbing Docker layers from the registry, because the specified commit _(ae885c0a6226)_ has already been pulled. Singularity Docker cache is located in `$HOME/.singularity/docker`.

In [9]:
singularity pull --size 3000 --name "jupyter-ext.img" docker://jupyter/base-notebook

Initializing Singularity image subsystem
Opening image file: jupyter-ext.img
Creating 3000MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-ext.img
Docker image path: index.docker.io/jupyter/base-notebook:latest
Cache folder set to /home/vagrant/.singularity/docker
Importing: base Singularity environment
Importing: /home/vagrant/.singularity/docker/sha256:e0a742c2abfd5e2a6f8ed15b1c78e873cf9559b96a04204daf6de5df01e3124c.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:486cb8339a27635fa93dc47aa0c689326a0a7cce388966d16daf8d265436cf7f.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:dc6f0d824617ad8a5d1163a5b2084814665dd83156317ad06ccf14deb517a053.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:4f7a5649a30e3f318ce5d7e4dbcbbeb6c0938c4cbae4d4a641fe910562ff4978.tar.gz
Importing: /home/vagrant/.singularity/docker/sha256:672363445ad2c734e29221a6b47f4e614b5adc8a3cdca3364f62db2ed2bdff0c.tar.gz
Importing: /home/vagrant/.singulari

In [3]:
singularity exec -e jupyter-ext.img jupyter -h

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the jupyter command's version and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json

Available subcommands: bundlerextension kernelspec lab labextension labhub
migrate nbconvert nbextension notebook run serverextension troubleshoot trust


### Installing software _(the quick way)_
By default Singularity containers mounted as read-only volumes, which means you won't be able to add content or install software _(even as a privileged user)_, save for default or system-mounted paths. In order to add content you must run your Singularity command with the `--writable` flag.

For an interactive shell into your container, use the `shell` subcommand. The command below also passes the `-e` flag, which tells Singularity to strip the host environment before entering the container.

In [ ]:
sudo singularity shell -e --writable jupyter-ext.img

Singularity: Invoking an interactive shell within container...



Alternatively, you can use the `exec` subcommand to execute commands in your container without leaving your host environment.

In [10]:
singularity exec -e --writable jupyter-ext.img /opt/conda/bin/conda install -y matplotlib
singularity exec -e --writable jupyter-ext.img /opt/conda/bin/conda install -y seaborn

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    cycler:           0.10.0-py36_0 conda-forge
    dbus:             1.10.22-0     conda-forge
    expat:            2.2.1-0       conda-forge
    fontconfig:       2.12.1-4      conda-forge
    freetype:         2.7-1         conda-forge
    gettext:          0.19.7-1      conda-forge
    glib:             2.51.4-0      conda-forge
    gst-plugins-base: 1.8.0-0       conda-forge
    gstreamer:        1.8.0-2       conda-forge
    icu:              58.1-1        conda-forge
    jpeg:             9b-1          conda-forge
    libiconv:         1.14-4        conda-forge
    libpng:           1.6.28-0      conda-forge
    libxcb:           1.12-1        conda-forge
    libxml2:          2.9.5-0       conda-forge
    matplotlib:       2.0.2-py36_2  conda-forge
    mkl:              2017.0.3-0    defaults   
    nu

Now seaborn is installed in your image.

In [11]:
singularity exec -e jupyter-ext.img conda list | grep seaborn

seaborn                   0.8.1                    py36_0    conda-forge


### Installing Software _(the reproducible way)_
Shelling into your container and making ad-hoc changes is excellent for debugging and initial development, but it is considered bad practice as the steps needed to construct your software environment are not captured and cannot be reproduced.

To make durable, reproducible changes you need to build a spec file from which you can bootstrap your container. Bootstrapping must be done by a privileged user

In [19]:
cat jupyter-bootstrapped.def

BootStrap: docker
From: jupyter/base-notebook

%environment
  export PATH=/opt/conda/bin:$PATH

%post
  export PATH=/opt/conda/bin:$PATH
  echo "Installing seaborn..."
  conda install matplotlib
  conda install seaborn


In [17]:
singularity create --force --size 2500 jupyter-bootstrapped.img
sudo /usr/local/bin/singularity bootstrap jupyter-bootstrapped.img jupyter-bootstrapped.def

Initializing Singularity image subsystem
Opening image file: jupyter-bootstrapped.img
Creating 2500MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-bootstrapped.img
Sanitizing environment
Building from bootstrap definition recipe
Adding base Singularity environment to container
Docker image path: index.docker.io/jupyter/base-notebook:latest
Cache folder set to /root/.singularity/docker
Exploding layer: sha256:e0a742c2abfd5e2a6f8ed15b1c78e873cf9559b96a04204daf6de5df01e3124c.tar.gz
Exploding layer: sha256:486cb8339a27635fa93dc47aa0c689326a0a7cce388966d16daf8d265436cf7f.tar.gz
Exploding layer: sha256:dc6f0d824617ad8a5d1163a5b2084814665dd83156317ad06ccf14deb517a053.tar.gz
Exploding layer: sha256:4f7a5649a30e3f318ce5d7e4dbcbbeb6c0938c4cbae4d4a641fe910562ff4978.tar.gz
Exploding layer: sha256:672363445ad2c734e29221a6b47f4e614b5adc8a3cdca3364f62db2ed2bdff0c.tar.gz
Exploding layer: sha256:b337aaee648d9f87e96fae8b24ae2dd887a2ded309b38dbee691fcdb040878cc.ta

In [18]:
singularity exec -e jupyter-bootstrapped.img conda list | grep seaborn

seaborn                   0.8.1                    py36_0    conda-forge


## Using your environment in a notebook
This next section will cover basic strategies for using your very new, very custom software environment in a Jupyter Notebook.

### Custom kernels
IPython notebooks interface with the system via an abstraction called _Kernels_. A wide variety of languages are supported via Kernels, and they can be customized by editing the kernelspec JSON file that defines them. Here is the default Python 3 kernelspec for reference:
```json
"argv": [
  "python",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "Python 3",
 "language": "python"
}
```

The `argv` key in this JSON object is the list that Jupyter uses to construct the kernel command when a notebook is started.

Remember the `singularity exec` subcommand? We can leverage that here to start a kernel in our container from a notebook server running in our host environment. All we need to do is prepend the components of the exec command to the `argv` list:
```json
"argv": [
  "singularity",
  "exec",
  "-e",
  "jupyter-bootstrapped.img",
  "python",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "Python 3",
 "language": "python"
}
```

#### Generating a new kernel
We'll start by generating a new kernelspec in a temporary location:

In [21]:
ipython kernel install --prefix /tmp

[InstallIPythonKernelSpecApp] WARNING | Installing to /tmp/share/jupyter/kernels, which is not in ['/home/vagrant/.local/share/jupyter/kernels', '/home/vagrant/venv-jupyter/share/jupyter/kernels', '/usr/local/share/jupyter/kernels', '/usr/share/jupyter/kernels', '/home/vagrant/.ipython/kernels']. The kernelspec may not be found.
Installed kernelspec python3 in /tmp/share/jupyter/kernels/python3


#### Editing the kernel
Now edit your kernelspec. An example can be found in this repo at <a href="/user/vagrant/edit/singularity-jupyter-demo/singularity-kernel.json">singularity-kernel.json</a>. Make sure to rename the kernelspec directory to avoid conflicts with existing kernels.

In [6]:
mv /tmp/share/jupyter/kernels/python3 /tmp/share/jupyter/kernels/seaborn
# Then edit /tmp/share/jupyter/kernels/seaborn/kernel.json (in our case we'll just copy the example)
cp singularity-kernel.json /tmp/share/jupyter/kernels/seaborn/kernel.json

#### Install the kernel
Finish by installing your new kernel to a location where your notebook will look when it starts. The `--user` flag specifies that you wish to install the kernel only for your user, and prevents the install from attempting to use `sys.prefix`.

In [7]:
jupyter kernelspec install --user /tmp/share/jupyter/kernels/seaborn

[InstallKernelSpec] Removing existing kernelspec in /home/vagrant/.local/share/jupyter/kernels/seaborn
[InstallKernelSpec] Installed kernelspec seaborn in /home/vagrant/.local/share/jupyter/kernels/seaborn
